In [1]:
# Importing libraries
import pandas as pd
import cdsapi
import xarray as xr
import numpy as np
import time

In [2]:
# Importing coordinates
local = pd.read_csv(r"C:\Users\joaoa\Desktop\Doutoramento\Download ERA5\local.cvs")
local

,point,latitude,longitude
0,0,42.2,-8.2
1,1,42.1,-8.6
2,2,42.1,-8.5
3,3,42.1,-8.4
4,4,42.1,-8.3
...,...,...,...
1007,1058,37.1,-7.8
1008,1059,37.1,-7.7
1009,1062,37.0,-8.0
1010,1063,37.0,-7.9


In [3]:
# Creating columns
col_list = ["time"]

for row in range(0, len(local)):
    col_list.append('P' + (4-len(str(local.iloc[row,0])))*'0' + str(local.iloc[row,0]))

    
# Cleaning data and filtering dates
Y = range(1950, 2024) #1950-2024
P = range(0, 1012) #0-1012
df = pd.DataFrame()

for year in Y: 
    i = time.time()
    y1 = df
    
    ds = xr.open_dataset(r"D:\Precipitation\by_civil_year\Ficheiros NC\era5land_" + str(year) + ".nc")
    df = ds.to_dataframe()
    df = df.reset_index()
    df["longitude"] = round(df["longitude"], 1)
    df["latitude"] = round(df["latitude"], 1)
    df["point"] = -1
    
    for row in range(0, len(local)):
        df.loc[(df["latitude"] == local.loc[row, "latitude"]) & (df["longitude"] == local.loc[row, "longitude"]), "point"] = local.loc[row, "point"]

    df = df[df["point"] >= 0]
    df = df.sort_values(by=["point", "time"])
    df = df.rename(columns={"tp": "precipitation"})
    df = df[["point", "latitude", "longitude", "time", "precipitation"]]
    
    if year == 1950:
        df = df[(df["time"].dt.month >= 10) & ((df["time"].dt.month > 10) | (df["time"].dt.day > 1) | (df["time"].dt.hour > 0))]
    elif year == 2023:
        df = df[(df["time"].dt.month < 10) | ((df["time"].dt.month == 10) & (df["time"].dt.day == 1) & (df["time"].dt.hour == 0))]
    else:
        df = df[(df["time"].dt.month != 2) | (df["time"].dt.day != 29)]
        
    df = df.reset_index(drop = True)
    y2 = df
    
    
    # Creating and saving water years
    if year > 1950:
        
        df_wy = pd.DataFrame(index = range(0, 8760), columns = col_list)
        df_wy.iloc[:2207, 0] = y1.iloc[-2207:, 3]
        df_wy.iloc[-6553:, 0] = y2.iloc[:6553, 3]
        
        for point in P:
            df_wy.iloc[:2207, point+1] = y1.loc[y1["point"] == local.iloc[point, 0]].iloc[-2207:, 4]
            df_wy.iloc[-6553:, point+1] = y2.loc[y2["point"] == local.iloc[point, 0]].iloc[:6553, 4]
            
        for row in range(8759, -1, -1):
            if df_wy.iloc[row, 0].hour == 1:
                df_wy.iloc[row, 1:] = df_wy.iloc[row, 1:]*1000
            else:
                df_wy.iloc[row, 1:] = (df_wy.iloc[row, 1:] - df_wy.iloc[row-1, 1:])*1000
        
        
        df_wy.to_csv(r"D:\Precipitation\by_water_year\era5land_total_precipitation_" + str(year-1) + "_" + str(year) + "_(mm).csv", compression = "zip", index = False)
        print("Water year " + str(year-1) + "_" + str(year) + " downloaded. Time: " + str(int(time.time()-i)) + " seconds.")

df_wy

Water year 1950_1951 downloaded. Time: 193 seconds.
Water year 1951_1952 downloaded. Time: 217 seconds.
Water year 1952_1953 downloaded. Time: 200 seconds.
Water year 1953_1954 downloaded. Time: 179 seconds.
Water year 1954_1955 downloaded. Time: 209 seconds.
Water year 1955_1956 downloaded. Time: 208 seconds.
Water year 1956_1957 downloaded. Time: 204 seconds.
Water year 1957_1958 downloaded. Time: 201 seconds.
Water year 1958_1959 downloaded. Time: 215 seconds.
Water year 1959_1960 downloaded. Time: 210 seconds.
Water year 1960_1961 downloaded. Time: 210 seconds.
Water year 1961_1962 downloaded. Time: 204 seconds.
Water year 1962_1963 downloaded. Time: 194 seconds.
Water year 1963_1964 downloaded. Time: 189 seconds.
Water year 1964_1965 downloaded. Time: 179 seconds.
Water year 1965_1966 downloaded. Time: 190 seconds.
Water year 1966_1967 downloaded. Time: 153 seconds.
Water year 1967_1968 downloaded. Time: 104 seconds.
Water year 1968_1969 downloaded. Time: 105 seconds.
Water year 1

,time,P0000,P0001,P0002,P0003,P0004,P0005,P0006,P0007,P0008,...,P1047,P1048,P1055,P1056,P1057,P1058,P1059,P1062,P1063,P1064
0,2022-10-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-10-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-10-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-10-01 04:00:00,0.001293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-10-01 05:00:00,0.001289,0.0,0.0,0.0,0.0,0.001293,0.001293,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-09-30 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8756,2023-09-30 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8757,2023-09-30 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8758,2023-09-30 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
